In [ ]:
!pip install roboflow
!pip install ultralytics

In [9]:
# Импорт библиотек
from roboflow import Roboflow
from ultralytics import YOLO
import json

In [ ]:
# Получение и подготовка датасета для обучения модели
rf = Roboflow(api_key="091mcn4Yvn3gWaa9FCeq")
project = rf.workspace("parkinglot-64itp").project("parking_lot-7nsmw")
version = project.version(19)
dataset = version.download("yolov8")

In [ ]:
# Загружаем предобученную модель для обучения 
model = YOLO("yolov8n.pt")

# Тренировка
model.train(data="Parking_Lot-19/data.yaml", epochs=20, imgsz=640)  # У меня ушло полтора часа

In [3]:
# Загрузка твоих весов
model = YOLO("runs/detect/train9/weights/best.pt")

# Тест на одной картинке
results = model.predict("6.png", save=True, imgsz=640, show_labels=False, show_conf=False)


image 1/1 C:\Users\1\_JupyterNotebook_content\_ParkingML\6.png: 480x640 57 emptylots, 4 moving cars, 14 nonemptylots, 105.3ms
Speed: 7.6ms preprocess, 105.3ms inference, 9.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict10


In [44]:
video_path  = r"C:\Users\1\Downloads\Telegram Desktop\парковка.avi"
output_json = "parking_boxes.json"

# Открываем видео и читаем первый кадр 
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
cap.release()
if not ret:
    raise RuntimeError(f"Не удалось прочитать кадр из {video_path}")

# Запускаем детектор на numpy-массиве кадра 
model   = YOLO("runs/detect/train9/weights/best.pt")
results = model(frame)[0]

# Определяем ID класса "moving car" для фильтрации 
moving_id = None
for cls_id, cls_name in model.names.items():
    if cls_name.lower() == "moving car":
        moving_id = cls_id
        break
if moving_id is None:
    raise ValueError("Класс 'moving car' не найден в model.names")

# Собираем только bbox для emptylot и nonemptylot 
boxes = []
for box in results.boxes:
    cid = int(box.cls[0].item())
    if cid == moving_id:
        continue  # пропускаем все движущиеся машины
    x1, y1, x2, y2 = box.xyxy[0].tolist()
    boxes.append([x1, y1, x2 - x1, y2 - y1])

# Сохраняем JSON с одним полем "boxes" 
with open(output_json, "w", encoding="utf-8") as f:
    json.dump({"boxes": boxes}, f, ensure_ascii=False, indent=2)

print(f"Сохранено {len(boxes)} боксов (без moving car) в {output_json}")



0: 480x640 58 emptylots, 3 moving cars, 14 nonemptylots, 63.8ms
Speed: 3.7ms preprocess, 63.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Сохранено 72 боксов (без moving car) в parking_boxes.json


In [45]:
video_path  = r"C:\Users\1\Downloads\Telegram Desktop\парковка.avi"
input_json  = "parking_boxes.json"
out_image   = "last_frame_with_boxes.png"

# Загружаем и валидируем JSON
with open(input_json, "r", encoding="utf-8") as f:
    data = json.load(f)

if "boxes" not in data:
    raise ValueError("Неверный формат JSON: ожидалось поле 'boxes'")

boxes = data["boxes"]

# Берём последний кадр из видео
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count - 1)
ret, last_frame = cap.read()
cap.release()
if not ret:
    raise RuntimeError("Не удалось прочитать последний кадр видео")

# Отрисовываем все bbox
for box in boxes:
    x, y, w_box, h_box = map(int, box)
    cv2.rectangle(
        last_frame,
        (x, y),
        (x + w_box, y + h_box),
        (0, 255, 0),  # зелёный цвет
        thickness=2
    )

# Сохраняем результат
cv2.imwrite(out_image, last_frame)
print(f"Результат сохранён в {out_image}")


Результат сохранён в last_frame_with_boxes.png
